In [2]:
from anthropic import Anthropic
from dotenv import load_dotenv
import json

In [3]:
load_dotenv()

client = Anthropic()

with open("three-dog-bakery-llc-order-on-tro.txt", "rb") as f:
    file_content = f.read()

MODEL_NAME = "claude-3-5-haiku-latest"

In [37]:
### Prompts

# System Prompt assigning role of paralegal
system = """
You are an experienced, thorough, and detail-oriented paralegal at a law firm. Your job is to support attorneys in your firm, who have exacting specifications.
"""

# Uses thinking tags to extract list of citations in brief
prompt_1 = f"""
Included below is a legal brief:
<brief>
{brief}
</brief>

Please carefully review the brief and provide me a list of every case, court rule, or statute that is cited in the brief. Before answering, you should double-check your work to make sure you have not missed a case or included one that is not present.
Please put your double-checked work in <double_check> tags and your final list of cases in <final_list> tags.
All cases should go in <case> tags, all statutes in <statute> tags, and all court rules in <rule> tags.
Finally, include the number of cases, statutes, and court rules you found in <count> tags. Please include only the number and nothing else.
"""

# Extract PIN citations out of case and "teach" model how a case citation is contsructed.
## Will need similar "lessons" for statutes and court rules. May want to run this in version 2 / write lessons as separate variable in new prompt
prompt_2 = f"""
<next_request>
I would now like you to make changes to the citation format of the cases in <final_list>. For background, cases are published in "reporters," and the citation usually have the following structure:
<caption>, <reporter_volume> <reporter_abbreviation> <starting_page_citation>, <pin_citation> (<court> <year>).

You should note the following subtleties:
1) The pin citation is optional, and is typically used to refer to a specific quote or point made in the case.
2) If a case is decided by the highest court (typically the Supreme Court) in the federal or state jurisdiction and it is obvious from the name of the reporter which jurisdiction the case is from, the court is not listed.
</next_request>

<examples>
For example:

In Brown v. Board of Education, 347 U.S. 483, 490 (1954):
-The caption is "Brown v. Board of Education"
-The reporter volume is number 347
-The reporter abbreviation is "U.S." (referring to the U.S. reports, which reports cases from the U.S. Supreme Court)
-The case starts on page 483 of volume 347 of the U.S. reports
-The pin citation is to page 490
-No court is listed because this case is from the U.S. Supreme Court
-The opinion was published in 1954

In Rollins v. Elwood, 141 Ill. 2d. 244 (1990):
-The caption is "Rollins v. Elwood"
-The reporter volume is number 141
-The reporter abbreviation is "Ill. 2d" (referring to the second Illinois reporter, which reports cases from the Illinois Supreme Court)
-The case starts on page 244 of volume 141 of the second Illinois reporter
-There is no pin ciation
-No court is listed because this case is from the Illinois Supreme Court
-The opinion was published in 1990

Alternatively, Rollins v. Elwood, 565 N.E.2d 1302 (Ill. 1990):
-The caption is "Rollins v. Elwood"
-The reporter volume is number 565
-The reporter abbreviation is "N.E.2d" (referring to the second Northeast reporter)
-The case starts on page 1302 of volume 565 of the second Northeast reporter
-There is no pin ciation
-The abbreviation "Ill." indicates this case is Illinois Supreme Court
-The opinion was published in 1990

The two prior cases are the same, but have slightly different citation formats due to the differences in the reporter.
</examples

<task>
You should now take the list of cases in <final_list> and remove any pin citations. Return any statutes and court rules in the same format as before.
</task>
"""

# Use tool call to get JSON list of cases
prompt_3 = """
Now use the structured_case_citation tool.
"""

In [38]:
### Tools

# Right now, only includes feature for extracting main case citation.
tools = [
        {
            "name": "structured_case_citation", # Extracts relevant features from main case caption
            "description": "Extracts elements of each case citation in <case> tags.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "cases": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "caption": {"type": "string", "description": "The case caption."},
                                "reporter": {"type": "string", "description": "The volume, name, and starting page number of the reporter"}, # Write in update for PIN citations
                                "court": {"type": "string", "description": "The name of the court issuing the opinion."},
                                "year": {"type": "string", "description": "The year the opinion was issued."} 
                                },
                            "required": ["caption", "reporter", "court", "year"]
                            }
                        }
                    },
                "required": ["cases"]
            }
        }
]

In [46]:
#Helper function to call JSON tool at end of extraction control flow
def call_json(message):
    for content in message.content:
        if content.type == "tool_use" and content.name == "structured_case_citation":
            return content.input

# A control flow to extract cases as JSON with single function call
def extract_cases_to_json(brief, tools=tools):
    #Helper function to make API calls
    def make_api_call(messages, use_tools=False):
        kwargs = {
            "model": MODEL_NAME,
            "max_tokens": 1024,
            "system": system,
            "messages": messages
        }
        if use_tools:
            kwargs["tools"] = tools
        return client.messages.create(**kwargs)

    #Initialize conversation history
    print("Intializing conversation")
    messages = []
    
    #Step 1: Extract citations
    print("Reviewing brief")
    messages.append({"role": "user", "content": prompt_1})
    response_1 = make_api_call(messages)
    messages.append({"role": "user", "content": response_1.content[0].text})
    
    #Step 2: Remove PIN cites
    print("Removing PIN cites")
    messages.append({"role": "user", "content": prompt_2})
    response_2 = make_api_call(messages)
    messages.append({"role": "user", "content": response_2.content[0].text})
    
    #Third message calls JSON tool
    print("Getting your list of cases")
    messages.append({"role": "user", "content": prompt_3})
    response_3 = make_api_call(messages, use_tools=True)

    # Uses JSON helper function to extract tool_use output
    return call_json(response_3)

In [47]:
extract_cases_to_json(file_content)

Intializing conversation
Reviewing brief
Removing PIN cites
Getting your list of cases


{'cases': [{'caption': 'Mazurek v. Armstrong',
   'reporter': '520 U.S. 968',
   'court': 'U.S.',
   'year': '1997'},
  {'caption': 'S.B. McLaughlin & Co. v. Tudor Oaks Condo. Project',
   'reporter': '877 F.2d 707',
   'court': '8th Cir.',
   'year': '1989'},
  {'caption': 'Phelps-Roper v. Nixon',
   'reporter': '509 F.3d 480',
   'court': '8th Cir.',
   'year': '2007'},
  {'caption': 'Dataphase Sys. Inc. v. CL Sys., Inc.',
   'reporter': '640 F.2d 109',
   'court': '8th Cir.',
   'year': '1981'},
  {'caption': 'Noodles Dev., LP. v. Ninth St. Partners, LLP',
   'reporter': '507 F. Supp. 2d 1030',
   'court': 'E.D. Mo.',
   'year': '2007'},
  {'caption': 'Iowa Utils. Bd. v. F.C.C.',
   'reporter': '109 F.3d 418',
   'court': '8th Cir.',
   'year': '1996'},
  {'caption': 'Local Union No. 884, United Rubber, Cork, Linoleum, & Plastic Workers of Am. v. Bridgestone / Firestone, Inc.',
   'reporter': '61 F.3d 1347',
   'court': '8th Cir.',
   'year': '1995'},
  {'caption': 'Watkins Inc. v. 